In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(".env")

# 手动设置环境变量
# os.environ["OPENAI_API_BASE_URL"] = "https://ai-yyds.com/v1"
# os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXX0bDb"
# os.environ["MODEL_NAME"] = "test"

## 文档处理预制链（Document Processing Prebuilt Chains）
在 LangChain 中，文档处理预制链（Document Processing Prebuilt Chains）主要用于处理和操作文档数据，以便进行后续的语言模型（LLM）处理。

### `load_summarize_chain` (加载摘要链)
+ 功能：
  + 用于对文档进行摘要提取。
  + 它支持多种摘要提取方法，例如 stuff、map_reduce、refine 和 map_rerank。
+ 使用方法：
  + 首先，你需要加载文档数据，并将其转换为 LangChain 的 Document 对象。
  + 然后，你可以使用 load_summarize_chain 函数加载摘要链，并指定所需的摘要提取方法。
  + 最后，你可以调用链的 run 方法，传入文档列表，以生成摘要。

#### 提取方法：stuff
stuff（填充）
+ 含义：
  + 这是最简单的方法。它将所有文档内容“填充”到一个提示（prompt）中，然后一次性传递给语言模型（LLM）进行摘要。
+ 特点：
  + 简单快捷，适用于文档内容较短的情况。
  + 当文档内容过长时，可能会超出 LLM 的上下文窗口限制，导致信息丢失或摘要效果不佳。
+ 适用场景：
  + 处理短篇文档或已知文档内容不会超过 LLM 上下文窗口的情况。

最常见的文档链，将文档直接塞进prompt中，为LLM回答问题提供上下文资料，适合小文档场景

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI

prompt_template = """对以下文字做简洁的总结:
{text}
简洁的总结:"""

prompt = PromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)
llm_chain = LLMChain(llm=llm, prompt=prompt)

stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="text",
)

loader = PyPDFLoader("./sources/loader.pdf")
docs = loader.load()

print(stuff_chain.run(docs))

In [ ]:
# 使用预封装好的load_summarize_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

loader = PyPDFLoader("./sources/loader.pdf")
docs = loader.load()

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    verbose=True,
)

chain.run(docs)

#### 提取方法：map_reduce
map_reduce（映射归约）
+ 含义：
  + 该方法将文档分割成较小的块，然后对每个块分别进行摘要（映射阶段）。
  + 最后，将所有块的摘要合并成最终的摘要（归约阶段）。
+ 特点：
  + 适用于处理大型文档，可以有效地处理超出 LLM 上下文窗口的内容。
  + 摘要质量可能不如 stuff 方法，因为每个块的摘要都是独立的，可能会丢失一些上下文信息。
+ 适用场景：
  + 处理长篇文档、书籍、报告等。

Map reduce先将每个文档或文档块分别投喂给LLM，并得到结果集（Map步骤），然后通过一个文档合并链，获得一个输出结果（Reduce步骤）
![Alt Text](./images/map.png)
![Alt Text](./images/reduce.png)

In [ ]:
from langchain.chains import MapReduceDocumentsChain
from langchain.chains import ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# load pdf
loader = PyPDFLoader("./sources/loader.pdf")
docs = loader.load()

# split text
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=0,
)
split_docs = text_splitter.split_documents(docs)
# print(split_docs)

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# map chain
map_template = """对以下文字做简洁的总结:
"{content}"
简洁的总结:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(
    llm=llm,
    prompt=map_prompt,
)

# reduce chain
reduce_template = """以下是一个摘要集合:
{doc_summaries}
将上述摘要与所有关键细节进行总结.
总结:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(
    llm=llm,
    prompt=reduce_prompt,
)

stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries",
)

reduce_final_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,
    # 超过4000个token就会切入到下一个stuff_chain
    collapse_documents_chain=stuff_chain,
    token_max=4000,
)

# map reduce chain
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="content",
    reduce_documents_chain=reduce_final_chain,
)

In [ ]:
summary = map_reduce_chain.run(split_docs)
print(summary)

#### 提取方法：refine
refine（精炼）
+ 含义：
  + 该方法首先对第一个文档块生成摘要，然后将后续文档块与之前的摘要一起传递给 LLM，逐步“精炼”摘要。
  + 这种方法保留了更多的上下文信息。
+ 特点：
  + 相比 map_reduce 方法，可以生成更高质量的摘要。
  + 处理速度较慢，因为需要多次调用 LLM。
+ 适用场景：
  + 需要高质量摘要，且可以容忍较长处理时间的情况。

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

# 加载PDF文档
loader = PyPDFLoader("./sources/loader.pdf")
docs = loader.load()

# 文档分割split
text_split = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 1000,
    chunk_overlap=0
)
split_docs = text_split.split_documents(docs)

prompt_template = """对以下文字做简洁的总结:
{text}
简洁的总结:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "你的任务是产生最终摘要\n"
    "我们已经提供了一个到某个特定点的现有回答:{existing_answer}\n"
    "我们有机会通过下面的一些更多上下文来完善现有的回答(仅在需要时使用).\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根据新的上下文，用中文完善原始回答.\n"
    "如果上下文没有用处,返回原始回答."
)
refine_prompt = PromptTemplate.from_template(refine_template)

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="documents",
    output_key="output_text",
)

In [ ]:
result = chain({"documents":split_docs}, return_only_outputs=True)
print(result["output_text"])

In [ ]:
print("\n\n".join(result["intermediate_steps"][:3]))

#### 提取方法：map_rerank
map_rerank（映射重排序）
+ 含义：
  + 该方法首先对每个文档块生成摘要（映射阶段）。
  + 然后，它会尝试评估哪些摘要是最相关的（重新排序阶段）。
  + 最后，它会返回得分最高的摘要。
+ 特点：
  + 此方法需要它返回单个最佳摘要。
  + 依赖于LLM评估文档的相关性。
+ 适用场景：
  + 当你需要从一组文档中找到最相关的摘要时。

先将每个文档或文档块投喂给LLM,并对每个文档或文档块生成问题的答案进行打分，然后将打分最高的文档或文档块作为最终答案返回

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    temperature=0
)

# load
loader = PyPDFLoader("./sources/loader.pdf")
docs = loader.load()

# split
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

chain = load_qa_with_sources_chain(
    llm=llm,
    chain_type="map_rerank", 
    # 元数据增加source字段
    metadata_keys=['source'], 
    # 打印中间执行步骤
    return_intermediate_steps=True
)

In [ ]:
print(chain)

In [ ]:
query = "what is this document talk about?answer by chinese"
result = chain({"input_documents":split_docs,"question":query})
result

### RetrievalQA (检索问答链)
+ 功能：
  + 用于从文档中检索相关信息，并回答用户的问题。
  + 它结合了文档检索和问答生成功能。
+ 使用方法：
  + 首先，你需要加载文档数据，并创建一个向量存储（Vector Store），用于存储文档的嵌入向量。
  + 然后，你需要创建一个检索器（Retriever），用于从向量存储中检索相关文档。
  + 最后，你可以使用 RetrievalQA 类创建检索问答链，并调用其 run 方法，传入用户的问题，以生成答案。

#### 1

#### 1

### MapReduceDocumentsChain (MapReduce文档链)
+ 功能：
  + 用于处理大型文档，将其分割成小块，并并行处理。
  + 它支持 MapReduce 模式，可以提高处理效率。
+ 使用方法：
  + 首先，你需要加载文档数据，并将其分割成小块。
  + 然后，你需要创建 MapReduce 文档链，并指定所需的 LLM 和提示模板。
  + 最后，你可以调用链的 run 方法，传入文档块列表，以生成最终结果。
+ 使用场景：
  + 当文档过大，无法一次性加载到内存中，可以使用该链。
  + 需要并行处理文档时，可以使用该链。